<a href="https://colab.research.google.com/github/ModernOctave/Thermal-Image-Human-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Raw Dataset
Download and extract the raw dataset

In [2]:
!wget -v https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
!unzip -q FLIR_ADAS_v2.zip -d FLIR_ADAS_v2

--2022-07-15 09:50:37--  https://adas-dataset-v2.flirconservator.com/dataset/full/FLIR_ADAS_v2.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)... 54.182.0.78, 54.182.0.101, 54.182.0.56, ...
Connecting to adas-dataset-v2.flirconservator.com (adas-dataset-v2.flirconservator.com)|54.182.0.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11865443849 (11G) [application/zip]
Saving to: ‘FLIR_ADAS_v2.zip’

FLIR_ADAS_v2.zip    100%[===================>]  11.05G  20.3MB/s    in 9m 22s  

2022-07-15 10:00:00 (20.2 MB/s) - ‘FLIR_ADAS_v2.zip’ saved [11865443849/11865443849]



# Install Dependancies


In [ ]:
# Import Tensorflow 2.0
%tensorflow_version 2.x

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

In [2]:
import tensorflow as tf

# Create datasets

In [1]:
from preprocessing import label_dataset, split_dataset, DataGen

labeled_dataset = label_dataset(dataset_path="FLIR_ADAS_v2/images_thermal_train/")
train_dataset, validation_dataset = split_dataset(labeled_dataset, 0.8, shuffle=True)

train_gen = DataGen(train_dataset, batch_size=32)
validation_gen = DataGen(validation_dataset, batch_size=32)

2022-07-16 12:34:55.457696: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-16 12:34:55.461900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-16 12:34:55.461933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


loading annotations into memory...
Done (t=1.23s)
creating index...
index created!
labeling the dataset...


100%|██████████| 10742/10742 [00:00<00:00, 106215.95it/s]


# Model


## Create the model

In [5]:
def build_cnn_model():
    cnn_model = tf.keras.Sequential([
                                     
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu, input_shape=(512, 640, 3)),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),

        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(64, activation=tf.nn.relu),

        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    
    return cnn_model
  
cnn_model = build_cnn_model()

# Initialize the model by passing some data through
# cnn_model.predict(train_images[[0]])

# Compile the model
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the summary of the layers in the model.
print(cnn_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 510, 638, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 255, 319, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 253, 317, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 126, 158, 64)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1274112)           0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [26]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S

base = EfficientNetV2S(include_top=True)

inputs = tf.keras.Input(shape=(512, 640, 3))
x = tf.keras.layers.Resizing(384, 384)(inputs)
x = base(x, training=False)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[tf.keras.metrics.BinaryAccuracy()])

# print(model.summary())

## Train the model

In [ ]:
cnn_model = tf.keras.models.load_model("model.h5")

In [27]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience=2, restore_best_weights=True)]

model.fit(train_gen, epochs=10, validation_data=validation_gen, callbacks=callbacks)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
cnn_model.save('model.h5')

## Test the model


In [ ]:
loss, accuracy = cnn_model.evaluate(test_images, test_labels, batch_size=1)